Working on adding a lot of features just to see if it can get the score up regardless of how complicated or where the data is coming from

Features that I will be adding

* Taget given surface counts
* words from the target and source document
  * possible back prop into these vectors, idea is to replace tf-idf with some nn and back prop here
* using a linear layer near the output to combine the features

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer, SimpleAverageLayer
from wordvecs import WordVectors, EmbeddingLayer, WordTokenizer
import json
import re
import random

theano.config.floatX = 'float32'
#theano.config.linker = 'cvm_nogc'
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 20000

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
with open('/data/matthew/external-wiki2.json') as f:
    queries = json.load(f)['queries']

In [4]:
len(queries)

9915

In [5]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

8917

In [6]:
wordvectors = WordVectors(
    fname="/data/matthew/enwiki-20141208-pages-articles-multistream-links7-output1.bin",
    redir_fname='/data/matthew/enwiki-20141208-pages-articles-multistream-redirect7.json',
    negvectors=False,
    sentence_length=200,
)
wordvectors.add_unknown_words = False

In [7]:
# with open('/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json') as f:
#     page_redirects = json.load(f)
page_redirects = wordvectors.redirects

In [8]:
len(wordvectors.vectors)

4056055

In [9]:
with open('/data/matthew/enwiki-20141208-pages-articles-multistream-surface-counts7.json') as f:
    surface_counts = json.load(f)

In [10]:
# try and make the surfaces items match what we are looking for
surface_counts_re = re.compile('([\.,!\?])')
for sk in surface_counts.keys():
    nsk = sk.replace('(', '-lrb-').replace(')', '-rrb-')
    nsk = surface_counts_re.sub(' \\1', nsk)
    if nsk != sk:
        surface_counts[nsk] = surface_counts[sk]

In [11]:
from wikireader import WikiRegexes, WikipediaReader

In [12]:
def PreProcessedQueries(wikipedia_dump_fname, vectors=wordvectors, queries=queries, redirects=page_redirects, surface=surface_counts):
    
    get_words = re.compile('[^a-zA-Z0-9 ]')
    get_link = re.compile('.*?\[(.*?)\].*?')
    
    wordvec = WordTokenizer(vectors, sentence_length=200)
    documentvec = WordTokenizer(vectors, sentence_length=1)
    
    queried_pages = set()
    for docs, q in queries.iteritems():
        wordvec.tokenize(docs)
        for sur, v in q.iteritems():
            wrds_sur = get_words.sub(' ', sur)
            wordvec.tokenize(wrds_sur)
            link_sur = get_link.match(sur).group(1)
            wordvec.tokenize(link_sur)
            for link in v['vals'].keys():
                wrds = get_words.sub(' ', link)
                wordvec.tokenize(wrds)
                tt = WikiRegexes.convertToTitle(link)
                documentvec.get_location(tt)
                queried_pages.add(tt)

    added_pages = set()
    for title in queried_pages:
        if title in redirects:
            #wordvec.tokenize(self.redirects[title])
            documentvec.get_location(redirects[title])
            added_pages.add(redirects[title])
    queried_pages |= added_pages

    page_content = {}

#     class GetWikipediaWords(WikipediaReader, WikiRegexes):

#         def readPage(ss, title, content):
#             tt = ss.convertToTitle(title)
#             if tt in queried_pages:
#                 cnt = ss._wikiToText(content)
#                 page_content[tt] = wordvec.tokenize(cnt)

#     GetWikipediaWords(wikipedia_dump_fname).read()
    
    rr = redirects
    rq = queried_pages
    rc = page_content
    rs = surface

    class PreProcessedQueriesCls(object):
        
        wordvecs = wordvec
        documentvecs = documentvec
        queries = queries
        redirects = rr
        queried_pages = rq
        page_content = rc
        surface_counts = rs
        
        
    return PreProcessedQueriesCls

In [13]:
basePreProcessedQueries = PreProcessedQueries('/data/matthew/enwiki-20141208-pages-articles-multistream.xml')

In [14]:
len(wordvectors.vectors)

4056055

In [34]:
for qu in queries.values():
    for en in qu.values():
        en['boosted'] = 1.0

In [16]:
def cleanUpMultipleLinks():
    for qu in queries.values():
        for en in qu.values():
            gold_page = en['gold']
            gold_title = WikiRegexes.convertToTitle(gold_page)
            pages = set()
            for p in en['vals'].keys():
                wiki_title = WikiRegexes.convertToTitle(p)
                if wiki_title == gold_title and p != en['gold']:
                    del en['vals'][p]
                elif p not in pages:
                    pages.add(p)
                else:
                    del en['vals'][p]
cleanUpMultipleLinks()

In [17]:
sum(len(q) for q in queries)

34863863

In [18]:
def removeSingleLinkTargets():
    "items that the surface link set only has a single item are trival, so remove"
    get_link = re.compile('.*?\[(.*?)\].*?')
    for quk, qu in queries.items():
        for sur in qu.keys():
            surlink = get_link.match(sur).group(1)
            surmatch = surlink.lower()
            surcounts = surface_counts.get(surmatch)
            if surcounts and len(surcounts) == 1:
                # this is trival since there is only one item
                del qu[sur]
        if not qu:
            # we removed all the links on this page, remove it otherwise the program crashes
            del queries[quk]
removeSingleLinkTargets()

In [19]:
sum(len(q) for q in queries)

34793912

In [35]:
class EntityVectorLinkExp(basePreProcessedQueries):

    batch_size = 1000 #20000
    num_training_items = 500000 #200000

    def __init__(self):
        self.sentence_length = self.wordvecs.sentence_length
        self.document_length = 100
        
        self.num_words_to_use_conv = 5
        self.enable_boosting = False
        self.num_negative_target_samples = 1
        #self.enable_match_surface = False
        #self.enable_link_counts = True
        self.enable_train_wordvecs = False
        self.enable_cap_boosting = True

        self._setup()

    def _setup(self):
        #self.x_document_input = T.imatrix('x_doc')

        #self.x_document_id = T.ivector('x_doc_id')
        self.x_surface_text_input = T.imatrix('x_surface_link')
        self.x_surface_context_input = T.imatrix('x_surface_cxt')  # TODO

        self.x_target_input = T.ivector('x_target')
        self.x_target_words = T.imatrix('x_target_words')
        self.x_matches_surface = T.ivector('x_match_surface')
        self.x_matches_counts = T.imatrix('x_matches_counts')
        self.x_link_id = T.ivector('x_link_id')

        #self.y_score = T.vector('y')
        self.y_answer = T.ivector('y_ans')  # contains the location of the gold answer so we can compute the loss
        self.y_grouping = T.imatrix('y_grouping')
        self.y_boosted = T.vector('y_boosted')


        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix().astype(theano.config.floatX))
        self.embedding_W_docs = theano.shared(self.documentvecs.get_numpy_matrix().astype(theano.config.floatX))
        
#         self.document_l = lasagne.layers.InputLayer(
#             (None,self.document_length),
#             input_var=self.x_document_input
#         )

#         self.document_embedding_l = EmbeddingLayer(
#             self.document_l,
#             W=self.embedding_W,
#             add_word_params=False,
#         )

#         self.document_conv1_l = lasagne.layers.Conv2DLayer(
#             self.document_embedding_l,
#             num_filters=500,
#             filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
#             name='document_conv1',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_max_l = lasagne.layers.Pool2DLayer(
#             self.document_conv1_l,
#             name='document_pool1',
#             pool_size=(self.document_length - self.num_words_to_use_conv, 1),
#             mode='sum',
#         )

#         self.document_dens1 = lasagne.layers.DenseLayer(
#             self.document_max_l,
#             num_units=250,
#             name='doucment_dens1',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_drop1 = lasagne.layers.DropoutLayer(
#             self.document_dens1,
#             p=.25,
#         )

#         document_output_length = 200

#         self.document_dens2 = lasagne.layers.DenseLayer(
#             self.document_drop1,
#             num_units=225,
#             name='document_dens2',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_drop2 = lasagne.layers.DropoutLayer(
#             self.document_dens2,
#             p=.25,
#         )

#         self.document_dens3 = lasagne.layers.DenseLayer(
#             self.document_drop2,
#             num_units=document_output_length,
#             name='document_dens3',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_output = lasagne.layers.get_output(self.document_dens3)

        self.surface_context_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_context_input,
        )

        self.surface_context_embedding_l = EmbeddingLayer(
            self.surface_context_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )

        self.surface_context_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_context_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_cxt_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

#         self.surface_context_avg1_l = SimpleAverageLayer(
#             [self.surface_context_conv1_l, self.surface_context_l],
#             #name='surface_context_avg'
#         )

        self.surface_context_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_context_conv1_l,
            name='surface_cxt_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )

        self.surface_input_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_text_input
        )

        self.surface_embedding_l = EmbeddingLayer(
            self.surface_input_l,
            W=self.embedding_W,
            add_word_params=self.enable_train_wordvecs,
        )

        self.surface_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

#         self.surface_avg1_l = SimpleAverageLayer(
#             [self.surface_conv1_l, self.surface_input_l],
#             #name='surface_avg'
#         )

        self.surface_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_conv1_l,
            name='surface_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )

        self.surface_merged_l = lasagne.layers.ConcatLayer(
            [self.surface_context_pool1_l, self.surface_pool1_l]
        )

        self.surface_dens1 = lasagne.layers.DenseLayer(
            self.surface_merged_l,
            name='surface_dens1',
            num_units=250,
            nonlinearity=lasagne.nonlinearities.tanh,
        )

#         self.surface_drop1 = lasagne.layers.DropoutLayer(
#             self.surface_dens1,
#             p=.25,
#         )

#         self.surface_dens2 = lasagne.layers.DenseLayer(
#             self.surface_drop1,
#             name='surface_dens2',
#             num_units=200,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_aligned_l = InputLayer(
#             (None, document_output_length),
#             input_var=self.document_output[self.x_document_id,:]
#         )

        ##############################################
        ## changed to not use the documented

#         self.source_l = lasagne.layers.ConcatLayer(
#             [self.document_aligned_l, self.surface_dens1]
#         )

        self.source_dens1 = lasagne.layers.DenseLayer(
            self.surface_dens1,   # CHANGED
            num_units=300,
            name='source_dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.source_drop1 = lasagne.layers.DropoutLayer(
            self.source_dens1,
            p=.25,
        )

        self.source_dens12 = lasagne.layers.DenseLayer(
            self.source_drop1,
            num_units=250,
            name='source_dens12',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.source_drop12 = lasagne.layers.DropoutLayer(
            self.source_dens12,
            p=.25,
        )

        compared_vector_size = self.wordvecs.vector_size #+ 2 # extra space for if it matches the surface text

        self.source_dens2 = lasagne.layers.DenseLayer(
            self.source_drop12,
            num_units=compared_vector_size,  # this is the same size as the learned wikipedia vectors
            name='source_dens2',
            nonlinearity=lasagne.nonlinearities.linear,
        )

        self.source_out = lasagne.layers.get_output(self.source_dens2)

        matched_surface_reshaped = self.x_matches_surface.reshape(
            (self.x_matches_surface.shape[0], 1, 1, 1)).astype(theano.config.floatX)

        self.target_input_l = lasagne.layers.InputLayer(
            (None,),
            input_var=self.x_target_input
        )

        self.target_matched_surface_input_l = lasagne.layers.InputLayer(
            (None,1,1,1),
            input_var=matched_surface_reshaped,
        )
        
        self.target_matched_counts_input_l = lasagne.layers.InputLayer(
            (None,5),
            input_var=self.x_matches_counts.astype(theano.config.floatX),
        )

        self.target_embedding_l = EmbeddingLayer(
            lasagne.layers.reshape(self.target_input_l, ([0], 1)),
            W=self.embedding_W_docs,
            add_word_params=False,
        )

        self.target_combined_feats_l = lasagne.layers.ConcatLayer(
            [self.target_embedding_l, self.target_matched_surface_input_l,
            lasagne.layers.reshape(self.target_matched_counts_input_l, ([0],1,1,[1]))],
            axis=3
        )

#         self.target_words_input_l = lasagne.layers.InputLayer(
#             (None,self.sentence_length),
#             input_var=self.x_target_words,
#         )

#         self.target_words_embedding_l = EmbeddingLayer(
#             self.target_words_input_l,
#             W=self.embedding_W,
#             add_word_params=False,
#         )

#         self.target_words_conv1_l = lasagne.layers.Conv2DLayer(
#             self.target_words_embedding_l,
#             name='target_wrds_conv1',
#             filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
#             num_filters=350,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.target_words_pool1_l = lasagne.layers.Pool2DLayer(
#             self.target_words_conv1_l,
#             name='target_wrds_pool1',
#             pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
#             mode='sum',
#         )

#         self.target_merge_l = lasagne.layers.ConcatLayer(
#             [lasagne.layers.reshape(self.target_words_pool1_l, ([0], [1])),
#              lasagne.layers.reshape(self.target_embedding_l, ([0], [3]))]
#         )

#         self.target_dens1 = lasagne.layers.DenseLayer(
#             self.target_merge_l,
#             name='target_wrds_dens1',
#             num_units=400,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.target_drop1 = lasagne.layers.DropoutLayer(
#             self.target_dens1,
#             p=.25,
#         )

#         self.target_dens2 = lasagne.layers.DenseLayer(
#             self.target_drop1,
#             name='target_wrds_dens1',
#             num_units=compared_vector_size,
#             nonlinearity=lasagne.nonlinearities.linear,
#         )

        self.target_simple = lasagne.layers.DenseLayer(
            self.target_embedding_l, #self.target_combined_feats_l,
            name='target_simple1',
            num_units=compared_vector_size,
            nonlinearity=lasagne.nonlinearities.linear,
        )

#         self.target_dens1 = lasagne.layers.DenseLayer(
#             self.target_conv1_l,
#             name='target_dens1',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.target_drop1 = lasagne.layers.DropoutLayer(
#             self.target_dens1,
#             p=.25,
#         )

#         self.target_dens2 = lasagne.layers.DenseLayer(
#             self.target_drop1,
#             name='target_dens2',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )



        #self.target_out = lasagne.layers.get_output(self.target_embedding_l)


#         self.target_out = T.concatenate(
#             [self.embedding_W[self.x_target_input],
#              matched_surface_reshaped,
#             1-matched_surface_reshaped],
#              axis=1)


        #self.target_out = self.embedding_W[self.x_target_input]
        #self.target_out = lasagne.layers.get_output(self.target_dens2)

        self.target_out = lasagne.layers.get_output(self.target_simple)

        # compute the cosine distance between the two layers
        self.source_aligned_l = self.source_out[self.x_link_id, :]

        # this uses scan internally, which means that it comes back into python code to run the loop.....fml
        self.dotted_vectors =  T.batched_dot(self.target_out, self.source_aligned_l)
        # diag also does not support a C version.........
        #self.dotted_vectors = T.dot(self.target_out, self.source_aligned_l.T).diagonal()

        def augNorm(v):
            return T.maximum(T.basic.pow(T.basic.pow(T.basic.abs_(v), 2).sum(axis=1) + .001, .5), .001)

        self.res_l = self.dotted_vectors / (augNorm(self.target_out) * augNorm(self.source_aligned_l) + .001)
        
        self.res_cap = T.clip((T.tanh(self.res_l) + 1) / 2, .001, .999)
        
        #############################
        ## Linear features combined
        #############################
        
        
        self.linear_features_combined = lasagne.layers.concat(
            [lasagne.layers.InputLayer(
                    (None, 1), 
                    input_var=self.res_l.reshape((self.res_l.shape[0], 1)),
                ),
            lasagne.layers.reshape(self.target_matched_surface_input_l, ([0],1)),
            self.target_matched_counts_input_l],
            axis=1
        )
        
        self.linear_features_dens_l = lasagne.layers.DenseLayer(
            self.linear_features_combined,
            nonlinearity=lasagne.nonlinearities.linear,
            num_units=1,
            name='linear_final_l',
            W=lasagne.init.Normal(mean=1.0),
        )
        
        self.linear_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.linear_features_dens_l, ([0],))
        )
        
        ########################################
        ## true output values
        ########################################
        
        self.true_output = self.linear_output
        
        
        
#         self.res_l = self.dotted_vectors / ((self.target_out.norm(1, axis=1) + .001) *
#                                             (self.source_aligned_l.norm(1, axis=1) + .001))


        #self.golds = self.res_cap[self.y_answer]

#         def maxOverRange(indx):
#             #return T.max(self.res_cap[T.arange(indx[0],indx[1])]) - self.res_cap[indx[2]]
#             #return -( self.res_l[indx[2]] - T.log(T.exp(self.res_l[T.arange(indx[0],indx[1])]).sum()) )
#             return -( self.res_l[indx[2]] - self.res_l[indx[0]])

#         # build a tensor to make a matrix with one set on each dimention
#         self.grouped, grouped_update = theano.scan(maxOverRange, sequences=self.y_grouping)

        def setSubSelector(indx, outputs):
            return T.set_subtensor(outputs[T.arange(indx[0], indx[1]), indx[3]], 1)

        num_target_samples = self.linear_output.shape[0]

        select_seq = T.concatenate([
            self.y_grouping,
            T.arange(self.y_grouping.shape[0]).reshape((self.y_grouping.shape[0], 1))
        ], axis=1)

        self.selecting_matrix, _ = theano.scan(
            setSubSelector,
            outputs_info=T.zeros((num_target_samples, self.y_grouping.shape[0])), #num_target_samples)),
            #n_steps=self.y_grouping.shape[0]
            sequences=select_seq,
        )

#         self.groupped_elems = T.dot(self.selecting_matrix[-1], 
#                                     T.diag(T.exp(self.true_output)))
#         self.groupped_res = T.log(self.groupped_elems.sum(axis=0)[T.arange(self.y_grouping.shape[0])])
        self.groupped_elems = T.dot(self.selecting_matrix[-1].T, 
                                   T.exp(self.true_output))
        self.groupped_res = T.log(self.groupped_elems)
        
        self.loss_vec = self.groupped_res - self.true_output[self.y_grouping[:,2]]
        
        if self.enable_boosting:
            self.loss_scalar = T.dot(self.y_boosted, self.loss_vec)
        else:
            self.loss_scalar = self.loss_vec.sum()

        self.all_params = (
            #lasagne.layers.get_all_params(self.target_dens2) +
            # TODO: add params for the target stuff,
            lasagne.layers.get_all_params(self.target_simple) +
            lasagne.layers.get_all_params(self.source_dens2) +
            lasagne.layers.get_all_params(self.linear_features_dens_l)
            #lasagne.layers.get_all_params(self.document_dens2)
        )

        # weight the positive samples more since there are fewer of them,
        # freaking hack
        #self.loss_vec = -(10 * self.y_score * T.log(self.res_cap) + (1.0 - self.y_score) * T.log(1.0 - self.res_cap))

        #self.loss_vec = T.nnet.binary_crossentropy(self.res_cap, self.y_score)

        #self.loss_vec = T.exp(T.max(self.res_cap - self.res_cap[self.y_answer] + .1, 0)) - 1  # TODO: maybe have some squared term here or something?

        # this one works reasonably well
        #self.loss_vec = - T.log((T.clip(self.res_cap[self.y_answer] - self.res_cap, -1.0, 0.4) + 1.0) / 1.5)

        #self.loss_vec = self.grouped

        #self.loss_vec = - T.log((T.clip(self.res_l[self.y_answer] - self.res_l, -40.0, 10.0) + 40.0) / 51.0)
        #self.loss_vec = T.max(self.res_l[self.y_answer] - self.res_l + .1, 0)

        self.updates = lasagne.updates.adadelta(self.loss_scalar / self.loss_vec.shape[0], self.all_params)

        self.func_inputs = [
            #self.x_document_input,
            self.x_surface_text_input, self.x_surface_context_input, #self.x_document_id,
            self.x_target_input, self.x_matches_surface, self.x_matches_counts, self.x_link_id,
            self.y_answer, self.y_grouping, self.y_boosted
        ]  # self.x_target_words,

        ################################################################3
        ## TODO: need to return the actual output layer instead of the res_cap, since that is something else now
        
        self.train_func = theano.function(
            self.func_inputs,
            [self.true_output, self.loss_vec.sum(), self.loss_scalar, self.loss_vec],
            updates=self.updates,
            on_unused_input='ignore',
        )

        self.test_func = theano.function(
            self.func_inputs,
            [self.true_output, self.loss_vec.sum(), self.loss_scalar, self.loss_vec],
            on_unused_input='ignore',
        )

    def reset_accums(self):
        self.current_documents = []
        self.current_surface_context = []
        self.current_surface_link = []
        self.current_link_id = []
        self.current_target_input = []
        self.current_target_words = []
        self.current_target_matches_surface = []
        self.current_target_id = []
        self.current_target_goal = []
        self.current_learning_groups = []
        self.learning_targets = []
        self.current_surface_target_counts = []
        self.current_boosted_groups = []
        
        self.failed_match = []
            
    def compute_batch(self, isTraining=True, useTrainingFunc=True):
        if isTraining and useTrainingFunc:
            func = self.train_func
        else:
            func = self.test_func
        self.reset_accums()
        self.total_links = 0
        self.total_loss = 0.0
        self.total_boosted_loss = 0.0

        get_words = re.compile('[^a-zA-Z0-9 ]')
        get_link = re.compile('.*?\[(.*?)\].*?')

        for doc, queries in self.queries.iteritems():
            # skip the testing documents while training and vice versa
            if queries.values()[0]['training'] != isTraining:
                continue
            docid = len(self.current_documents)
            self.current_documents.append(self.wordvecs.tokenize(doc, length=self.document_length))
            for surtxt, targets in queries.iteritems():
                self.current_link_id.append(docid)
                surid = len(self.current_surface_link)
                self.current_surface_context.append(self.wordvecs.tokenize(get_words.sub(' ' , surtxt)))
                surlink = get_link.match(surtxt).group(1)
                self.current_surface_link.append(self.wordvecs.tokenize(surlink))
                surmatch = surlink.lower()
                surcounts = self.surface_counts.get(surmatch)
                if not surcounts:
                    self.failed_match.append(surmatch)
                    surcounts = {}
                #target_page_input = []
                target_words_input = []
                target_matches_surface = []
                target_inputs = []
                target_learings = []
                target_match_counts = []
                target_gold_loc = -1
                target_group_start = len(self.current_target_input)
                
#                 pages = []
#                 pages_info_link = [] 
#                 gold = None
#                 # skip the items that we don't know the gold for
#                 if not targets['gold'] and isTraining:
#                     continue
#                 for target in set(targets['vals'].keys() +
#                                  random.sample(self.documentvecs.reverse_word_location, self.num_negative_target_samples)
#                                   ) - {None,}:
#                     isGold = target == targets['gold']
#                     wiki_title = WikiRegexes.convertToTitle(target)
#                     cnt = self.documentvecs.get_location(wiki_title)
#                     if wiki_title == 'nil':
#                         cnt = 0  # this is the stop symbol location
#                     if cnt is None:
#                         # were not able to find this wikipedia document
#                         # so just ignore tihs result since trying to train on it will cause
#                         # issues
#                         continue
#                     if isGold:
#                         gold = cnt
#                     if cnt not in pages:
#                         pages.append(cnt)
#                         pages_info_link.append((targets, target))
#                     elif isGold:
#                         # there are two links to the same page somehow
#                         # and we already have something that isn't linking to the gold
#                         for i in xrange(len(pages)):
#                             if pages[i] == cnt:
#                                 # this links to the same page as the gold
#                                 pages_info_link[i] = (targets, target)
#                 for i in xrange(len(pages)):
                    
                    
                for target in set(targets['vals'].keys() +
                                 random.sample(self.documentvecs.reverse_word_location, self.num_negative_target_samples)
                                  ) - {None,}:
                    
                    
                    isGold = target == targets['gold']
                    #cnt = self.page_content.get(WikiRegexes.convertToTitle(target))
                    wiki_title = WikiRegexes.convertToTitle(target)
                    cnt = self.documentvecs.get_location(wiki_title)
                    if wiki_title == 'nil':
                        cnt = 0  # this is the stop symbol location
                    if cnt is None:
                        # were not able to find this wikipedia document
                        # so just ignore tihs result since trying to train on it will cause
                        # issues
                        continue
                    if isGold:
                        target_gold_loc = len(target_inputs)
                    #target_page_input.append(cnt)
                    target_words_input.append(self.wordvecs.tokenize(get_words.sub(' ', target)))
                    target_inputs.append(cnt)  # page_content already tokenized
                    target_matches_surface.append(int(surmatch == target.lower()))
                    target_learings.append((targets, target))
                    target_match_counts.append(surcounts.get(wiki_title, 0))
                    #if wiki_title not in surcounts:
                    #    print surcounts, wiki_title
                if target_gold_loc is not None or not isTraining:  # if we can't get the gold item
                    # contain the index of the gold item for these items, so it can be less then it
                    gold_loc = (len(self.current_target_goal) + target_gold_loc)
                    sorted_match_counts = [-4,-3,-2,-1] + sorted(set(target_match_counts))
                    #print sorted_match_counts
                    target_match_counts_indicators = [
                        [
                            int(s == sorted_match_counts[-1]),
                            int(s == sorted_match_counts[-2]),
                            int(s == sorted_match_counts[-3]),
                            int(0 < s <= sorted_match_counts[-4]),
                            int(s == 0),
                        ]
                        for s in target_match_counts
                    ]
                    self.current_target_goal += [gold_loc] * len(target_inputs)
                    self.current_target_input += target_inputs
                    self.current_target_id += [surid] * len(target_inputs)
                    self.current_target_words += target_words_input   # TODO: add
                    self.current_target_matches_surface += target_matches_surface
                    self.current_surface_target_counts += target_match_counts_indicators
                    target_group_end = len(self.current_target_input)
                    self.current_learning_groups.append(
                        [target_group_start, target_group_end,
                         gold_loc])
                    self.current_boosted_groups.append(targets['boosted'])

                #self.current_target_goal.append(isGold)
                self.learning_targets += target_learings
            if len(self.current_target_id) > self.batch_size:
                self.run_batch(func)
                if self.total_links > self.num_training_items:
                    return self.total_loss / self.total_links, self.total_boosted_loss / self.total_links

        if len(self.current_target_id) > 0:
            self.run_batch(func)

        return self.total_loss / self.total_links, self.total_boosted_loss / self.total_links

    def run_batch(self, func):
        res_vec, loss_sum, loss_boosted, loss_vec = func(
            #self.current_documents,
            self.current_surface_link, self.current_surface_context, #self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_surface_target_counts, self.current_target_id,
            self.current_target_goal, self.current_learning_groups, self.current_boosted_groups,
            # self.current_target_words,
        )
        self.check_params()
        self.total_links += len(self.current_target_id)
        self.total_loss += loss_sum
        self.total_boosted_loss += loss_boosted
        learned_groups = []  # right...dict not hashable....
        for i in xrange(len(res_vec)):
            # save the results from this pass
            l = self.learning_targets[i]
            if l[1] in l[0]['vals']:
                l[0]['vals'][ l[1] ] = float(res_vec[i])
            if l[0] not in learned_groups:
                learned_groups.append(l[0])
        for group in learned_groups:
            if group['gold']:
                correct = max(group['vals']) == group['vals'][group['gold']]
                group['boosted'] *= .4 if correct else 2.0
                if self.enable_cap_boosting:
                    if group['boosted'] > 10:
                        group['boosted'] = 10.0
                    elif group['boosted'] < 0.1:
                        group['boosted'] = 0.1
        self.reset_accums()

    def check_params(self):
        if any([np.isnan(v.get_value(borrow=True)).any() for v in self.all_params]):
            raise RuntimeError('nan in some of the parameters')



queries_exp = EntityVectorLinkExp()

In [21]:
def evalCurrentState(trainingData=True, numSamples=50000):
    all_measured = 0
    all_correct = 0
    all_trained = 0
    for qu in queries.values():
        for en in qu.values():
            if en['training'] != trainingData:
                continue
            if en['gold']:
                if all_trained > numSamples:
                    break
                all_measured += 1
                all_trained += len(en['vals'].values())
                m = max(en['vals'].values())
                if en['vals'][en['gold']] == m and m != 0:
                    all_correct += 1
           
    r = all_measured, float(all_correct) / all_measured
    print r
    return r

In [22]:
def evalCurrentStateRank(trainingData=True, numSamples=50000):
    all_measured = 0
    all_correct_place = 0
    p_counts = dict((k,0) for k in range(0,10))
    all_trained = 0
    for qu in queries.values():
        for en in qu.values():
            if en['training'] != trainingData:
                continue
            if en['gold']:
                if all_trained > numSamples:
                    break
                svals = sorted(en['vals'].values(), key=lambda x: -x)
                gv = en['vals'][en['gold']]
                if gv == 0:
                    continue
                all_measured += 1
                for i in xrange(len(svals)):
                    if svals[i] == gv:
                        if i < 10:
                            p_counts[i] += 1
                        all_correct_place += i + 1
                        break

    r = all_measured, float(all_correct_place) / all_measured, p_counts
    print r
    return r


In [23]:
import random
def augmentTrainingData():
    for quk in queries.keys():
        qu = queries[quk]
        for enk in qu.keys():
            en = qu[enk]
            if not en['gold']:
                del qu[enk]
        if not qu:
            del queries[quk]
    for qu in queries.values():
        training = random.random() > .15
        for en in qu.values():
            en['training'] = training
augmentTrainingData()

In [24]:
def findWrongItems(trainingData=True, numSamples=50):
    ret = {}
    for qu in queries.values():
        for ek, en in qu.items():
            if en['training'] != trainingData:
                continue
            for e in en:
                if en['gold']:
                    if len(ret) > numSamples:
                        return ret
                    m = max(en['vals'].values())
                    g = en['vals'][en['gold']]
                    if g != m and g != 0:
                        ret[ek] = en
    return ret

In [36]:
queries_exp.check_params()

In [37]:
queries_exp.num_training_items = 50000

In [ ]:
%time print queries_exp.compute_batch()

In [ ]:
len(queries_exp.current_surface_target_counts)

In [ ]:
evalCurrentState(False, 500000)

In [ ]:
evalCurrentState(True, 500000)

In [ ]:
#exp_results = []

for i in xrange(15):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)
    if i % 5 == 4:
        exp_results.append(('testing run', queries_exp.compute_batch(False)))
        exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
        exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

In [65]:
queries_exp.num_training_items = 50000
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

(2742, 0.7268417213712619)


In [40]:
exp_results

[(0, (0.12738508756817421, 0.12738508756817421)),
 (1, (0.10150150461575834, 0.10150150461575834)),
 (2, (0.086746932930875098, 0.086746932930875098)),
 (3, (0.077410592491854113, 0.077410592491854113)),
 (4, (0.07112434892858023, 0.07112434892858023)),
 ('testing run', (0.072812150552051577, 0.072812150552051577)),
 ('training state', (2709, 0.6895533407161314)),
 ('testing state', (2742, 0.6699489423778264)),
 (5, (0.066095054831548747, 0.066095054831548747)),
 (6, (0.061728537496568557, 0.061728537496568557)),
 (7, (0.058245651796764053, 0.058245651796764053)),
 (8, (0.055652992978548851, 0.055652992978548851)),
 (9, (0.053185571698501667, 0.053185571698501667)),
 ('testing run', (0.068910807368774071, 0.068910807368774071)),
 ('training state', (2709, 0.7807308970099668)),
 ('testing state', (2742, 0.7155361050328227)),
 (10, (0.050894028007795598, 0.050894028007795598)),
 (11, (0.048983230974878257, 0.048983230974878257)),
 (12, (0.04691198698104658, 0.04691198698104658)),
 (13, (

In [ ]:
surface_counts['The Silent World of Nicholas Quinn'.lower()]

In [32]:
surface_counts['canada']

{u'1968_canadian_grand_prix': 2,
 u'1969_canadian_grand_prix': 5,
 u'1970_canadian_grand_prix': 2,
 u'1971_canadian_grand_prix': 3,
 u'1973_canadian_grand_prix': 2,
 u'1974_canadian_grand_prix': 2,
 u'1976_canadian_grand_prix': 1,
 u'1977_canadian_grand_prix': 2,
 u'1978_canadian_grand_prix': 3,
 u'1979_canadian_grand_prix': 1,
 u'1981_canadian_grand_prix': 2,
 u'1982_canadian_grand_prix': 3,
 u'1983_canadian_grand_prix': 1,
 u'1984_canadian_grand_prix': 7,
 u'1984_canadian_mens_rowing_eight': 1,
 u'1985_canadian_grand_prix': 4,
 u'1986_canadian_grand_prix': 3,
 u'1988_canadian_grand_prix': 3,
 u'1989_canadian_grand_prix': 9,
 u'1990_canadian_grand_prix': 2,
 u'1991_canadian_grand_prix': 7,
 u'1992_canadian_grand_prix': 5,
 u'1993_canadian_grand_prix': 5,
 u'1994_canadian_grand_prix': 5,
 u'1995_canadian_grand_prix': 3,
 u'1996_canadian_grand_prix': 2,
 u'1997_canadian_grand_prix': 7,
 u'1998_canadian_grand_prix': 2,
 u'1999_canadian_grand_prix': 5,
 u'2000_canadian_grand_prix': 3,
 u'

In [33]:
surface_counts['urban district']

{u'banstead': 1,
 u'chengqu_changzhi': 1,
 u'district__lrb_china_rrb_': 4,
 u'district__lrb_taiwan_rrb_': 1,
 u'districts_of_england': 1,
 u'districts_of_germany': 2,
 u'independent_cities_of_germany': 13,
 u'independent_city': 1,
 u'list_of_districts_of_vietnam': 23,
 u'list_of_urban_districts_of_vietnam': 7,
 u'stadtbezirk': 20,
 u'urban_area': 1,
 u'urban_district': 7,
 u'urban_district__lrb_great_britain_and_ireland_rrb_': 947,
 u'urban_districts_formed_in_england_and_wales_189495': 1,
 u'urban_districts_of_sweden': 1}

In [31]:
findWrongItems()

{u' [Canada] sent 26,791 troops to the war , with 7,000 more': {'boosted': 10.0,
  u'gold': u'Canada',
  u'training': True,
  u'vals': {u'-NIL-': -5.281156063079834,
   u'A1 Team Canada': 1.4902822971343994,
   u'Canada': 2.679443359375,
   u'Canada (New France)': 1.8414738178253174,
   u'Canada Masters': -2.293558120727539,
   u"Canada men's national ice hockey team": -1.1700072288513184,
   u"Canada men's national junior ice hockey team": -2.009016275405884,
   u"Canada men's national soccer team": 0,
   u'Canada national cricket team': 2.8703830242156982,
   u'Canada national junior hockey team': 0,
   u"Canada national men's ice hockey team": 0,
   u'Canada national rugby union team': 0.6203822493553162,
   u"Canada women's national soccer team": -2.6548800468444824,
   u'Canadian Open (tennis)': -1.6199089288711548,
   u'Canadian Recording Industry Association': -1.5224575996398926,
   u'Canadian Soccer Association': -4.040358066558838,
   u'Canadian cricket team': -0.661493062973

In [ ]:

exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

In [ ]:
exp_results

In [ ]:
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

In [ ]:
for i in xrange(4):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)

In [41]:
queries_exp.all_params[-2:]

[linear_final_l.W, linear_final_l.b]

In [63]:
queries_exp.all_params[-2:][0].get_value(borrow=True)[2] = 1

In [64]:
queries_exp.all_params[-2:][0].get_value(borrow=True)

array([[ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]], dtype=float32)

In [49]:
saved_lin_layer

array([[ 6.69999981],
       [ 2.27134681],
       [ 2.30811667],
       [ 0.90609312],
       [-0.10643651],
       [-0.70718431],
       [-1.42647231]], dtype=float32)

In [ ]:
findWrongItems()

In [ ]:
surface_counts['The Silent World of Nicholas Quinn'.lower()]

In [ ]:
%time for i in range(5): print queries_exp.compute_batch()

In [ ]:
queries.values()[0]

In [ ]:
queries_exp.all_params

In [ ]:
queries_exp.all_params[-2].get_value(borrow=True)

In [ ]:
queries_exp.all_params[-2].get_value(borrow=True)[0] = 20.0